In [16]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

In [18]:
X

,Beta_Left Frontal__sum_values,Beta_Left Frontal__abs_energy,Beta_Left Frontal__mean_abs_change,Beta_Left Frontal__mean_change,Beta_Left Frontal__mean_second_derivative_central,Beta_Left Frontal__median,Beta_Left Frontal__mean,Beta_Left Frontal__length,Beta_Left Frontal__standard_deviation,Beta_Left Frontal__variation_coefficient,Beta_Left Frontal__variance,Beta_Left Frontal__skewness,Beta_Left Frontal__kurtosis,Beta_Left Frontal__root_mean_square,Beta_Left Frontal__absolute_sum_of_changes,Beta_Left Frontal__longest_strike_below_mean,Beta_Left Frontal__longest_strike_above_mean,Beta_Left Frontal__count_above_mean,Beta_Left Frontal__count_below_mean,Beta_Left Frontal__last_location_of_maximum,Beta_Left Frontal__first_location_of_maximum,Beta_Left Frontal__last_location_of_minimum,Beta_Left Frontal__first_location_of_minimum,Beta_Left Frontal__maximum,Beta_Left Frontal__absolute_maximum,Beta_Left Frontal__minimum,Beta_Left Frontal__benford_correlation,Beta_Left Frontal__time_reversal_asymmetry_statistic__lag_1,Beta_Left Frontal__time_reversal_asymmetry_statistic__lag_2,Beta_Left Frontal__time_reversal_asymmetry_statistic__lag_3,Beta_Left Frontal__c3__lag_1,Beta_Left Frontal__c3__lag_2,Beta_Left Frontal__c3__lag_3,Beta_Left Frontal__cid_ce__normalize_True,Beta_Left Frontal__cid_ce__normalize_False,Beta_Left Frontal__symmetry_looking__r_0.05,Beta_Left Frontal__large_standard_deviation__r_0.1,Beta_Left Frontal__large_standard_deviation__r_0.15000000000000002,Beta_Left Frontal__large_standard_deviation__r_0.2,Beta_Left Frontal__large_standard_deviation__r_0.25,Beta_Left Frontal__quantile__q_0.1,Beta_Left Frontal__quantile__q_0.2,Beta_Left Frontal__quantile__q_0.3,Beta_Left Frontal__quantile__q_0.4,Beta_Left Frontal__quantile__q_0.6,Beta_Left Frontal__quantile__q_0.7,Beta_Left Frontal__quantile__q_0.8,Beta_Left Frontal__quantile__q_0.9,Beta_Left Frontal__autocorrelation__lag_0,Beta_Left Frontal__autocorrelation__lag_1,Beta_Left Frontal__autocorrelation__lag_2,Beta_Left Frontal__autocorrelation__lag_3,Beta_Left Frontal__autocorrelation__lag_4,Beta_Left Frontal__autocorrelation__lag_5,Beta_Left Frontal__autocorrelation__lag_6,Beta_Left Frontal__autocorrelation__lag_7,Beta_Left Frontal__autocorrelation__lag_8,Beta_Left Frontal__autocorrelation__lag_9,"Beta_Left Frontal__agg_autocorrelation__f_agg_""mean""__maxlag_40","Beta_Left Frontal__agg_autocorrelation__f_agg_""median""__maxlag_40","Beta_Left Frontal__agg_autocorrelation__f_agg_""var""__maxlag_40",Beta_Left Frontal__partial_autocorrelation__lag_1,Beta_Left Frontal__partial_autocorrelation__lag_2,Beta_Left Frontal__partial_autocorrelation__lag_3,Beta_Left Frontal__partial_autocorrelation__lag_4,Beta_Left Frontal__partial_autocorrelation__lag_5,Beta_Left Frontal__partial_autocorrelation__lag_6,Beta_Left Frontal__partial_autocorrelation__lag_7,Beta_Left Frontal__partial_autocorrelation__lag_8,Beta_Left Frontal__partial_autocorrelation__lag_9,Beta_Left Frontal__number_cwt_peaks__n_1,Beta_Left Frontal__number_cwt_peaks__n_5,Beta_Left Frontal__number_peaks__n_1,Beta_Left Frontal__number_peaks__n_3,Beta_Left Frontal__number_peaks__n_5,Beta_Left Frontal__number_peaks__n_10,Beta_Left Frontal__number_peaks__n_50,Beta_Left Frontal__binned_entropy__max_bins_10,Beta_Left Frontal__index_mass_quantile__q_0.1,Beta_Left Frontal__index_mass_quantile__q_0.2,...,"Gamma_Right Frontal__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""var""","Gamma_Right Frontal__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""max""","Gamma_Right Frontal__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""min""","Gamma_Right Frontal__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""mean""","Gamma_Right Frontal__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""var""","Gamma_Right Frontal__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","Gamma_Right Frontal__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","Gamma_Right Frontal__agg_linear_trend__attr_

In [17]:
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'

data_types = ['Wake','N1', 'N2', 'N3', 'REM']

loaded_data = {}

for data_type in data_types:
    X_y_groups = {}
    
    X = pd.read_hdf(load_path + data_type +  'six_second_freq_df.h5', key='df', mode='r')
    y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
    groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
    X, y , groups = X.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

    #Transform the X into TS_Fresh Features___
    # 1. Convert to TS_Fresh format Dataframe 
    ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))
    
    # 2. Extract TS_Fresh Features from the dataframe
    settings = EfficientFCParameters()
    extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)

    # 3. Asign extract_ts_fresh_df to the variable X
    X = extracted_ts_fresh_df.copy()
    #Drop columns where all values are NA 
    X = X.dropna(axis = 1)
    
    #Drop columns where all values are the same
    # Find columns where all values are the same
    same_value_columns = X.columns[X.nunique() == 1]
    # Drop columns with the same values
    X = X.drop(columns=same_value_columns)
    

    #Generate regional features dict per data type 
    #Generate a region to features dictionary - this will enable us to run expts regionally as before
    regional_features_dict = {}
    region_channel_dict = constants.region_to_channel_dict
    regions = list(region_channel_dict.keys())
    for region in regions:
        region_features = [col for col in X.columns if region + '_' in col]
        if len(region_features) > 0 : 
            regional_features_dict[region] = region_features
       

    X_y_groups['X'] = X
    X_y_groups['y'] = y
    X_y_groups['groups'] = groups
    X_y_groups['regions_dict'] = regional_features_dict

    loaded_data[data_type] = X_y_groups

04-Sep-23 14:41:24 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 160/160 [02:43<00:00,  1.02s/it]
04-Sep-23 14:44:18 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 158/158 [04:09<00:00,  1.58s/it]
04-Sep-23 14:48:41 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 158/158 [17:30<00:00,  6.65s/it]
04-Sep-23 15:06:25 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 160/160 [10:45<00:00,  4.04s/it]
04-Sep-23 15:17:27 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 157/157 [06:53<00:00,  2.63s/it]


## Save and Load Results with joblib

In [53]:
#Saving Results 
joblib.dump(expt_1_results, 'Results/expt_1_TS_fresh_efficient_feats_results.pkl')
joblib.dump(expt_1_combined_results, 'Results/expt_1_combined_TS_fresh_efficient_feats_results.pkl')
joblib.dump(expt_2_results, 'Results/expt_2_TS_fresh_efficient_feats_results.pkl')
joblib.dump(expt_3_results, 'Results/expt_3_TS_fresh_efficient_feats_results.pkl')
joblib.dump(expt_4_results, 'Results/expt_4_TS_fresh_efficient_feats_results.pkl')

['Results/expt_4_TS_fresh_efficient_feats_results.pkl']

In [13]:
#Loading Results
expt_1_results = joblib.load('Results/expt_1_TS_fresh_efficient_feats_results.pkl')
expt_1_combined_results = joblib.load('Results/expt_1_combined_TS_fresh_efficient_feats_results.pkl')
expt_2_results = joblib.load('Results/expt_2_TS_fresh_efficient_feats_results.pkl')
expt_3_results = joblib.load('Results/expt_3_TS_fresh_efficient_feats_results.pkl')
expt_4_results = joblib.load('Results/expt_4_TS_fresh_efficient_feats_results.pkl')

### Expt 1

In [4]:
#Run Expt 1 experiments 

t1 = time.time()

expt_1_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_1_results[data_type] = results_df

t2 = time.time()

t2-t1

KeyError: 'Wake'

In [ ]:
results_df = expt_1_results['N1']
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(res_df)

In [15]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_1_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.516667,0.55,0.566667,0.45,0.483333,0.5,0.433333,0.533333,0.516667,0.55,0.566667,0.566667,0.633333


N1


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.75,0.8,0.8,0.75,0.8,0.8,0.75,0.825,0.7875,0.7375,0.7625,0.7375,0.8125


N2


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.561111,0.55,0.523611,0.595833,0.595833,0.595833,0.583333,0.577778,0.630556,0.556944,0.531944,0.570833,0.523611


N3


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.630556,0.558333,0.641667,0.619444,0.633333,0.593056,0.531944,0.484722,0.568056,0.591667,0.594444,0.558333,0.608333


REM


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.6125,0.625,0.625,0.65,0.6375,0.65,0.6,0.6,0.625,0.575,0.6125,0.6,0.575


## Expt_1 Combining Features

In [46]:
#Run Expt 1 experiments 

t1 = time.time()

expt_1_combined_results = {}

# for data_type in data_types:
for data_type in ['N1']:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']

    regions = list(regional_features_dict.keys())

    combined_regions_features_dict = {}
    
    for i, region_1 in enumerate(regions):
        for region_2 in regions[i+1:]:
            new_key = region_1 + '_' + region_2
            combined_regions_features_dict[new_key] = regional_features_dict[region_1] + regional_features_dict[region_2]
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = combined_regions_features_dict, random_states = [1,2] )
    
    expt_1_combined_results[data_type] = results_df

t2 = time.time()

t2-t1

118.9073839187622

In [9]:
results_df = expt_1_combined_results['N1']
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(res_df)

,random_state,Prefrontal_Frontal_RF_acc,Prefrontal_Frontal_RF_std,Prefrontal_Left Frontal_RF_acc,Prefrontal_Left Frontal_RF_std,Prefrontal_Right Frontal_RF_acc,Prefrontal_Right Frontal_RF_std,Prefrontal_Central_RF_acc,Prefrontal_Central_RF_std,Prefrontal_Left Central_RF_acc,Prefrontal_Left Central_RF_std,Prefrontal_Right Central_RF_acc,Prefrontal_Right Central_RF_std,Prefrontal_Left Temporal_RF_acc,Prefrontal_Left Temporal_RF_std,Prefrontal_Right Temporal_RF_acc,Prefrontal_Right Temporal_RF_std,Prefrontal_Parietal_RF_acc,Prefrontal_Parietal_RF_std,Prefrontal_Left Parietal_RF_acc,Prefrontal_Left Parietal_RF_std,Prefrontal_Right Parietal_RF_acc,Prefrontal_Right Parietal_RF_std,Prefrontal_Occipital_RF_acc,Prefrontal_Occipital_RF_std,Frontal_Left Frontal_RF_acc,Frontal_Left Frontal_RF_std,Frontal_Right Frontal_RF_acc,Frontal_Right Frontal_RF_std,Frontal_Central_RF_acc,Frontal_Central_RF_std,Frontal_Left Central_RF_acc,Frontal_Left Central_RF_std,Frontal_Right Central_RF_acc,Frontal_Right Central_RF_std,Frontal_Left Temporal_RF_acc,Frontal_Left Temporal_RF_std,Frontal_Right Temporal_RF_acc,Frontal_Right Temporal_RF_std,Frontal_Parietal_RF_acc,Frontal_Parietal_RF_std,Frontal_Left Parietal_RF_acc,Frontal_Left Parietal_RF_std,Frontal_Right Parietal_RF_acc,Frontal_Right Parietal_RF_std,Frontal_Occipital_RF_acc,Frontal_Occipital_RF_std,Left Frontal_Right Frontal_RF_acc,Left Frontal_Right Frontal_RF_std,Left Frontal_Central_RF_acc,Left Frontal_Central_RF_std,Left Frontal_Left Central_RF_acc,Left Frontal_Left Central_RF_std,Left Frontal_Right Central_RF_acc,Left Frontal_Right Central_RF_std,Left Frontal_Left Temporal_RF_acc,Left Frontal_Left Temporal_RF_std,Left Frontal_Right Temporal_RF_acc,Left Frontal_Right Temporal_RF_std,Left Frontal_Parietal_RF_acc,Left Frontal_Parietal_RF_std,Left Frontal_Left Parietal_RF_acc,Left Frontal_Left Parietal_RF_std,Left Frontal_Right Parietal_RF_acc,Left Frontal_Right Parietal_RF_std,Left Frontal_Occipital_RF_acc,Left Frontal_Occipital_RF_std,Right Frontal_Central_RF_acc,Right Frontal_Central_RF_std,Right Frontal_Left Central_RF_acc,Right Frontal_Left Central_RF_std,Right Frontal_Right Central_RF_acc,Right Frontal_Right Central_RF_std,Right Frontal_Left Temporal_RF_acc,Right Frontal_Left Temporal_RF_std,Right Frontal_Right Temporal_RF_acc,Right Frontal_Right Temporal_RF_std,Right Frontal_Parietal_RF_acc,Right Frontal_Parietal_RF_std,Right Frontal_Left Parietal_RF_acc,Right Frontal_Left Parietal_RF_std,Right Frontal_Right Parietal_RF_acc,Right Frontal_Right Parietal_RF_std,Right Frontal_Occipital_RF_acc,Right Frontal_Occipital_RF_std,Central_Left Central_RF_acc,Central_Left Central_RF_std,Central_Right Central_RF_acc,Central_Right Central_RF_std,Central_Left Temporal_RF_acc,Central_Left Temporal_RF_std,Central_Right Temporal_RF_acc,Central_Right Temporal_RF_std,Central_Parietal_RF_acc,Central_Parietal_RF_std,Central_Left Parietal_RF_acc,Central_Left Parietal_RF_std,Central_Right Parietal_RF_acc,Central_Right Parietal_RF_std,Central_Occipital_RF_acc,Central_Occipital_RF_std,Left Central_Right Central_RF_acc,Left Central_Right Central_RF_std,Left Central_Left Temporal_RF_acc,Left Central_Left Temporal_RF_std,Left Central_Right Temporal_RF_acc,Left Central_Right Temporal_RF_std,Left Central_Parietal_RF_acc,Left Central_Parietal_RF_std,Left Central_Left Parietal_RF_acc,Left Central_Left Parietal_RF_std,Left Central_Right Parietal_RF_acc,Left Central_Right Parietal_RF_std,Left Central_Occipital_RF_acc,Left Central_Occipital_RF_std,Right Central_Left Temporal_RF_acc,Right Central_Left Temporal_RF_std,Right Central_Right Temporal_RF_acc,Right Central_Right Temporal_RF_std,Right Central_Parietal_RF_acc,Right Central_Parietal_RF_std,Right Central_Left Parietal_RF_acc,Right Central_Left Parietal_RF_std,Right Central_Right Parietal_RF_acc,Right Central_Right Parietal_RF_std,Right Central_Occipital_RF_acc,Right Central_Occipital_RF_std,Left Temporal_Right Temporal_RF_acc,Left Temporal_Right Temporal_RF_std,Left Tempo

In [10]:
(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T )

,Prefrontal_Frontal_RF_acc,Prefrontal_Left Frontal_RF_acc,Prefrontal_Right Frontal_RF_acc,Prefrontal_Central_RF_acc,Prefrontal_Left Central_RF_acc,Prefrontal_Right Central_RF_acc,Prefrontal_Left Temporal_RF_acc,Prefrontal_Right Temporal_RF_acc,Prefrontal_Parietal_RF_acc,Prefrontal_Left Parietal_RF_acc,Prefrontal_Right Parietal_RF_acc,Prefrontal_Occipital_RF_acc,Frontal_Left Frontal_RF_acc,Frontal_Right Frontal_RF_acc,Frontal_Central_RF_acc,Frontal_Left Central_RF_acc,Frontal_Right Central_RF_acc,Frontal_Left Temporal_RF_acc,Frontal_Right Temporal_RF_acc,Frontal_Parietal_RF_acc,Frontal_Left Parietal_RF_acc,Frontal_Right Parietal_RF_acc,Frontal_Occipital_RF_acc,Left Frontal_Right Frontal_RF_acc,Left Frontal_Central_RF_acc,Left Frontal_Left Central_RF_acc,Left Frontal_Right Central_RF_acc,Left Frontal_Left Temporal_RF_acc,Left Frontal_Right Temporal_RF_acc,Left Frontal_Parietal_RF_acc,Left Frontal_Left Parietal_RF_acc,Left Frontal_Right Parietal_RF_acc,Left Frontal_Occipital_RF_acc,Right Frontal_Central_RF_acc,Right Frontal_Left Central_RF_acc,Right Frontal_Right Central_RF_acc,Right Frontal_Left Temporal_RF_acc,Right Frontal_Right Temporal_RF_acc,Right Frontal_Parietal_RF_acc,Right Frontal_Left Parietal_RF_acc,Right Frontal_Right Parietal_RF_acc,Right Frontal_Occipital_RF_acc,Central_Left Central_RF_acc,Central_Right Central_RF_acc,Central_Left Temporal_RF_acc,Central_Right Temporal_RF_acc,Central_Parietal_RF_acc,Central_Left Parietal_RF_acc,Central_Right Parietal_RF_acc,Central_Occipital_RF_acc,Left Central_Right Central_RF_acc,Left Central_Left Temporal_RF_acc,Left Central_Right Temporal_RF_acc,Left Central_Parietal_RF_acc,Left Central_Left Parietal_RF_acc,Left Central_Right Parietal_RF_acc,Left Central_Occipital_RF_acc,Right Central_Left Temporal_RF_acc,Right Central_Right Temporal_RF_acc,Right Central_Parietal_RF_acc,Right Central_Left Parietal_RF_acc,Right Central_Right Parietal_RF_acc,Right Central_Occipital_RF_acc,Left Temporal_Right Temporal_RF_acc,Left Temporal_Parietal_RF_acc,Left Temporal_Left Parietal_RF_acc,Left Temporal_Right Parietal_RF_acc,Left Temporal_Occipital_RF_acc,Right Temporal_Parietal_RF_acc,Right Temporal_Left Parietal_RF_acc,Right Temporal_Right Parietal_RF_acc,Right Temporal_Occipital_RF_acc,Parietal_Left Parietal_RF_acc,Parietal_Right Parietal_RF_acc,Parietal_Occipital_RF_acc,Left Parietal_Right Parietal_RF_acc,Left Parietal_Occipital_RF_acc,Right Parietal_Occipital_RF_acc
0,0.8,0.775,0.775,0.7875,0.8,0.75,0.7875,0.7875,0.7625,0.7625,0.7375,0.7875,0.775,0.8125,0.8,0.8,0.775,0.75,0.8,0.75,0.7875,0.7875,0.7875,0.8125,0.8,0.8125,0.775,0.825,0.7625,0.75,0.7875,0.7875,0.7875,0.7625,0.825,0.775,0.775,0.7875,0.7125,0.7625,0.8125,0.7625,0.8125,0.775,0.8,0.7625,0.775,0.775,0.7625,0.7875,0.8125,0.8,0.825,0.7875,0.7875,0.8,0.8375,0.775,0.775,0.75,0.725,0.8,0.775,0.825,0.75,0.7875,0.7875,0.825,0.725,0.75,0.7625,0.775,0.725,0.725,0.725,0.75,0.7625,0.775


## Expt 2 

In [23]:
#Run Expt 2 experiments 

t1 = time.time()

expt_2_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_2_results[data_type] = results_df

t2 = time.time()

t2-t1

155.25202798843384

In [10]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_2_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.694643,0.589286,0.510714,0.585714,0.4625,0.376786,0.369643,0.380357,0.435714,0.526786,0.564286,0.498214,0.553571


N1


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.671111,0.672222,0.672222,0.642222,0.658889,0.682222,0.626667,0.754444,0.648889,0.682222,0.706667,0.627778,0.693333


N2


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.5,0.444444,0.492222,0.455556,0.55,0.471111,0.448889,0.444444,0.528889,0.435556,0.455556,0.442222,0.522222


N3


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.523333,0.512222,0.512222,0.491111,0.502222,0.511111,0.418889,0.47,0.462222,0.428889,0.431111,0.468889,0.524444


REM


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.524444,0.458889,0.424444,0.522222,0.49,0.496667,0.502222,0.563333,0.466667,0.477778,0.497778,0.361111,0.431111


### Expt 3

In [25]:
#Run Expt 3 experiments 

t1 = time.time()

expt_3_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3)

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier()} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_3_results[data_type] = results_df

t2 = time.time()

t2-t1

191.62625527381897

In [11]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_3_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.367424,0.240152,0.348485,0.312879,0.250758,0.249242,0.268182,0.188636,0.320455,0.276515,0.214394,0.243182,0.248485


N1


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.44381,0.415238,0.393333,0.472381,0.484762,0.485714,0.48619,0.493333,0.415238,0.43,0.478095,0.429048,0.388571


N2


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.360952,0.374762,0.283333,0.289048,0.401429,0.360476,0.416667,0.259524,0.338571,0.37381,0.389048,0.302857,0.338571


N3


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.261905,0.269048,0.277619,0.269524,0.310952,0.284762,0.27,0.241905,0.277619,0.185714,0.227619,0.235238,0.271429


REM


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.398901,0.412088,0.391209,0.41978,0.368681,0.325824,0.448352,0.427473,0.369231,0.368681,0.382967,0.37033,0.348352


### Expt 4

In [27]:
t1 = time.time()

expt_4_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4)

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier()} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_4_results[data_type] = results_df

t2 = time.time()

t2-t1

197.1443247795105

In [12]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_4_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.286364,0.294697,0.318939,0.384091,0.275758,0.336364,0.331061,0.240152,0.382576,0.311364,0.302273,0.322727,0.330303


N1


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.470952,0.359524,0.372857,0.359048,0.451905,0.507143,0.458571,0.41619,0.387619,0.444286,0.442857,0.374762,0.400952


N2


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.311429,0.317143,0.288571,0.274762,0.35381,0.429524,0.338095,0.260476,0.374762,0.338571,0.282857,0.297143,0.296667


N3


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.227619,0.248095,0.22,0.31,0.269524,0.215714,0.240476,0.262857,0.339048,0.216667,0.221905,0.270476,0.278095


REM


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.434066,0.42033,0.434615,0.419231,0.391209,0.398352,0.413187,0.440659,0.455495,0.447802,0.448901,0.41978,0.376374
